In [ ]:
import sys
sys.path.append('../')
import os
import collections
import random
from pathlib import Path
import logging
import shutil
import time
from packaging import version
from collections import defaultdict

from tqdm import tqdm
import numpy as np
import gzip
import torch
import torch.nn as nn
from torch.nn.parallel import DistributedDataParallel as DDP
import torch.distributed as dist
import torch.backends.cudnn as cudnn

from src.param import parse_args
from src.utils import LossMeter
from src.dist_utils import reduce_dict
from transformers import T5Tokenizer, T5TokenizerFast
from src.tokenization import P5Tokenizer, P5TokenizerFast
from src.pretrain_model import P5Pretraining

_use_native_amp = False
_use_apex = False

# Check if Pytorch version >= 1.6 to switch between Native AMP and Apex
if version.parse(torch.__version__) < version.parse("1.6"):
    from transormers.file_utils import is_apex_available
    if is_apex_available():
        from apex import amp
    _use_apex = True
else:
    _use_native_amp = True
    from torch.cuda.amp import autocast

from src.trainer_base import TrainerBase

import pickle

def load_pickle(filename):
    with open(filename, "rb") as f:
        return pickle.load(f)


def save_pickle(data, filename):
    with open(filename, "wb") as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)
        
import json

def load_json(file_path):
    with open(file_path, "r") as f:
        return json.load(f)
    
def ReadLineFromFile(path):
    lines = []
    with open(path,'r') as fd:
        for line in fd:
            lines.append(line.rstrip('\n'))
    return lines

def parse(path):
    g = gzip.open(path, 'r')
    for l in g:
        yield eval(l)

In [ ]:
class DotDict(dict):
    def __init__(self, **kwds):
        self.update(kwds)
        self.__dict__ = self
        
args = DotDict()

args.distributed = False
args.multiGPU = True
args.fp16 = True
args.train = "beauty"
args.valid = "beauty"
args.test = "beauty"
args.batch_size = 16
args.optim = 'adamw' 
args.warmup_ratio = 0.05
args.lr = 1e-3
args.num_workers = 4
args.clip_grad_norm = 1.0
args.losses = 'rating,sequential,explanation,review,traditional'
args.backbone = 't5-small' # small or base
args.output = 'snap/beauty-small'
args.epoch = 10
args.local_rank = 0

args.comment = ''
args.train_topk = -1
args.valid_topk = -1
args.dropout = 0.1

args.tokenizer = 'p5'
args.max_text_length = 512
args.do_lower_case = False
args.word_mask_rate = 0.15
args.gen_max_length = 64

args.weight_decay = 0.01
args.adam_eps = 1e-6
args.gradient_accumulation_steps = 1
args.continuous_embed = True
'''
Set seeds
'''
args.seed = 2022
torch.manual_seed(args.seed)
random.seed(args.seed)
np.random.seed(args.seed)

'''
Whole word embedding
'''
args.whole_word_embed = True

cudnn.benchmark = True
ngpus_per_node = torch.cuda.device_count()
args.world_size = ngpus_per_node

LOSSES_NAME = [f'{name}_loss' for name in args.losses.split(',')]
if args.local_rank in [0, -1]:
    print(LOSSES_NAME)
LOSSES_NAME.append('total_loss') # total loss

args.LOSSES_NAME = LOSSES_NAME

gpu = 0 # Change GPU ID
args.gpu = gpu
args.rank = gpu
print(f'Process Launching at GPU {gpu}')

torch.cuda.set_device('cuda:{}'.format(gpu))

comments = []
dsets = []
if 'toys' in args.train:
    dsets.append('toys')
if 'beauty' in args.train:
    dsets.append('beauty')
if 'sports' in args.train:
    dsets.append('sports')
comments.append(''.join(dsets))
if args.backbone:
    comments.append(args.backbone)
comments.append(''.join(args.losses.split(',')))
if args.comment != '':
    comments.append(args.comment)
comment = '_'.join(comments)

if args.local_rank in [0, -1]:
    print(args)

In [ ]:
def create_config(args):
    from transformers import T5Config, BartConfig

    if 't5' in args.backbone:
        config_class = T5Config
    else:
        return None

    config = config_class.from_pretrained(args.backbone)
    config.dropout_rate = args.dropout
    config.dropout = args.dropout
    config.attention_dropout = args.dropout
    config.activation_dropout = args.dropout
    config.losses = args.losses

    return config


def create_tokenizer(args):
    from transformers import T5Tokenizer, T5TokenizerFast
    from src.tokenization import P5Tokenizer, P5TokenizerFast

    if 'p5' in args.tokenizer:
        tokenizer_class = P5Tokenizer

    tokenizer_name = args.backbone
    
    if args.continuous_embed:
        datamaps = load_json(os.path.join('../data', args.train, 'datamaps.json'))
        user_extra_ids = len(datamaps['user2id']) + 1 # index start from 1
        item_extra_ids = len(datamaps['item2id']) + 1 
    else:
        user_extra_ids = 0
        item_extra_ids = 0
    tokenizer = P5Tokenizer.from_pretrained(
            args.backbone, 
            max_length=args.max_text_length, 
            do_lower_case=args.do_lower_case,
            user_extra_ids=user_extra_ids,
            item_extra_ids=item_extra_ids
            )

    print(tokenizer_class, tokenizer_name)
    
    return tokenizer


def create_model(model_class, config=None):
    print(f'Building Model at GPU {args.gpu}')

    model_name = args.backbone

    model = model_class.from_pretrained(
        model_name,
        config=config
    )
    return model

In [ ]:
config = create_config(args)

if args.tokenizer is None:
    args.tokenizer = args.backbone
    
tokenizer = create_tokenizer(args)

model_class = P5Pretraining
model = create_model(model_class, config)

model = model.cuda()
if 'p5' in args.tokenizer:
    model.resize_token_embeddings(tokenizer.vocab_size)
    
model.tokenizer = tokenizer

In [ ]:
# args.load = "../snap/beauty-small/Epoch10.pth"
args.load = "../snap/pretrain_item/Epoch30.pth"
# Load Checkpoint
from src.utils import load_state_dict, LossMeter, set_global_logging_level
from pprint import pprint

def load_checkpoint(ckpt_path):
    state_dict = load_state_dict(ckpt_path, 'cpu')
    results = model.load_state_dict(state_dict, strict=False)
    print('Model loaded from ', ckpt_path)
    pprint(results)

ckpt_path = args.load
print(torch.norm(torch.mean(model.get_input_embeddings().weight.data[:model.tokenizer.sp_model.get_piece_size()], 0)))

load_checkpoint(ckpt_path)
print(torch.norm(torch.mean(model.get_input_embeddings().weight.data[model.tokenizer.sp_model.get_piece_size() + model.tokenizer._extra_ids:model.tokenizer.sp_model.get_piece_size() + model.tokenizer._extra_ids+model.tokenizer._user_extra_ids], 0)))
print(torch.norm(torch.mean(model.get_input_embeddings().weight.data[model.tokenizer.sp_model.get_piece_size() + model.tokenizer._extra_ids -4:], 0)))

from src.all_amazon_templates import all_tasks as task_templates

In [ ]:
from torch.utils.data import DataLoader, Dataset, Sampler
from src.pretrain_data import get_loader
from evaluate.utils import rouge_score, bleu_score, unique_sentence_percent, root_mean_square_error, mean_absolute_error, feature_detect, feature_matching_ratio, feature_coverage_ratio, feature_diversity
from evaluate.metrics4rec import evaluate_all

In [ ]:
test_task_list = {'relate_inout_embeds': ['6-1'] # or '1-6'
}
test_sample_numbers = {'learn_title': 1, 'relate_inout_embeds': 1}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='train', 
        tokenizer = tokenizer,
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gt_ratings = []
pred_ratings = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        print(batch['source_text'][0])
        print('goal: ', batch['target_text'][0])
        print('res: ', results[0])
        


In [ ]:
test_task_list = {'learn_title': ['7-1'] # or '1-6'
}
test_sample_numbers = {'learn_title': 1, 'relate_inout_embeds': 1}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='train', 
        tokenizer = tokenizer,
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gt_ratings = []
pred_ratings = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        print(batch['source_text'][0])
        print('goal: ', batch['target_text'][0])
        print('res: ', results[0])

In [ ]:
test_task_list = {'learn_category': ['8-1'] # or '1-6'
}
test_sample_numbers = {'learn_title': 1, 'relate_inout_embeds': 1, 'learn_category': 1}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='train', 
        tokenizer = tokenizer,
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gts = []
preds = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        gts.extend(batch['target_text'])
        preds.extend(results)
        # print(batch['source_text'][0])
        # print('goal: ', batch['target_text'][0])
        # print('res: ', results[0])
accy = np.mean(np.asarray(gts) == np.asarray(preds))
print(accy)

In [ ]:
test_task_list = {'learn_brand': ['9-1'] # or '1-6'
}
test_sample_numbers = {'learn_title': 1, 'relate_inout_embeds': 1, 'learn_brand': 1}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='train', 
        tokenizer = tokenizer,
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gts = []
preds = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        gts.extend(batch['target_text'])
        preds.extend(results)
        print(batch['source_text'][0])
        print('goal: ', batch['target_text'][0])
        print('res: ', results[0])
accy = np.mean(np.asarray(gts) == np.asarray(preds))
print(accy)

In [25]:
test_task_list = {'learn_user': ['11-1'] # or '1-6'
}
test_sample_numbers = {'learn_title': 1, 'relate_inout_embeds': 1, 'learn_user': 1}

zeroshot_test_loader = get_loader(
        args,
        test_task_list,
        test_sample_numbers,
        split=args.test, 
        mode='train', 
        tokenizer = tokenizer,
        batch_size=args.batch_size,
        workers=args.num_workers,
        distributed=args.distributed
)
print(len(zeroshot_test_loader))

gts = []
preds = []
for i, batch in tqdm(enumerate(zeroshot_test_loader)):
    with torch.no_grad():
        results = model.generate_step(batch)
        gts.extend(batch['target_text'])
        preds.extend(results)
        print(batch['source_text'][0])
        print('goal: ', batch['target_text'][0])
        print('res: ', results[0])
accy = np.mean(np.asarray(gts) == np.asarray(preds))
print(accy)

66566
Data sources:  ['beauty']


KeyError: 'learn_price'